# Text classification starter

先读取文件，合并数据和target，分成train和test数据集。用算法训练，这里用到sklearn的naive bayes算法（MultinomialNB）来对train数据集进行分类，再用test数据集测试，得到正确率为92.3%。

需要改进的地方：
1. 加入semantics语义，尤其是对中文的，并用nltk
2. 分析为什么有92.3%
3. 研究其他算法的效果
4. 调参

-- alison


## Import libraries

In [1]:
# coding: utf-8 
#encoding=utf-8

import numpy as np
from dateutil import parser
import pandas as pd
from datetime import datetime,date,timedelta
import datetime, warnings, scipy
from dateutil import parser
import matplotlib as mpl
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV
import sklearn.datasets
from sklearn.model_selection import train_test_split

import json 
import csv 
import sys
import os

# import nltk
# nltk.download()


## Import data

In [2]:
def read_data(rootdir, target_name, target_index):
    
    for root, subdirs, files in os.walk(rootdir):
        for file in files:
            print(root+'/'+file)
            path = root+'/'+file
            with open(path, encoding="utf8") as f:
                input_file = f.read()
                data.append(input_file)
                target_names.append(target_name)
                target.append(target_index)

data = []
target_names = []
target = []
read_data("after/中性", "中性", 0)
read_data("after/买入", "买入", 1)
read_data("after/卖出", "卖出", 2)


after/中性/H3_AP202006141384842093_1.txt
after/中性/H3_AP202008311404492146_1.txt
after/中性/H3_AP202009011405122823_1.txt
after/中性/H3_AP202009011405242583_1.txt
after/买入/H3_AP202006101383773865_1.txt
after/买入/H3_AP202006101383776458_1.txt
after/买入/H3_AP202006101383776960_1.txt
after/买入/H3_AP202006101383781309_1.txt
after/买入/H3_AP202006101383781317_1.txt
after/买入/H3_AP202006101383784605_1.txt
after/买入/H3_AP202006101383784606_1.txt
after/买入/H3_AP202006101383786385_1.txt
after/买入/H3_AP202006101383788522_1.txt
after/买入/H3_AP202006101383788524_1.txt
after/买入/H3_AP202006101383788527_1.txt
after/买入/H3_AP202006101383789068_1.txt
after/买入/H3_AP202006101383802176_1.txt
after/买入/H3_AP202006101383802183_1.txt
after/买入/H3_AP202006101383808977_1.txt
after/买入/H3_AP202006101383809027_1.txt
after/买入/H3_AP202006101383836065_1.txt
after/买入/H3_AP202006101383836071_1.txt
after/买入/H3_AP202006101383836078_1.txt
after/买入/H3_AP202006101383836088_1.txt
after/买入/H3_AP202006101383848977_1.txt
after/买入/H3_AP20200610138

In [3]:
# combine as dataset
dataset = {
    "data" : data,
    "target_names" :target_names,
    "target" :target
}

# print keys
print(dataset.keys(),"\n")
# print all targets
print(dataset["target_names"])
# prints target set
set(dataset["target"]) 

dict_keys(['data', 'target_names', 'target']) 

['中性', '中性', '中性', '中性', '买入', '买入', '买入', '买入', '买入', '买入', '买入', '买入', '买入', '买入', '买入', '买入', '买入', '买入', '买入', '买入', '买入', '买入', '买入', '买入', '买入', '买入', '买入', '买入', '买入', '买入', '买入', '买入', '买入', '买入', '买入', '买入', '买入', '买入', '买入', '买入', '买入', '买入', '买入', '买入', '买入', '买入', '买入', '买入', '买入', '买入', '买入', '买入', '卖出', '卖出', '卖出', '卖出', '卖出', '卖出', '卖出', '卖出', '卖出', '卖出']


{0, 1, 2}

## Train / test data split

In [4]:
data_x = dataset["data"]
data_y = dataset["target"]

train_x, test_x, train_y, test_y = \
    train_test_split(data_x, data_y, test_size = 0.2, random_state = 0)

## Data exploration

### healper functions

In [5]:
# audit dataframe
def table_info(data):
    tab_info=pd.DataFrame(data.dtypes).T.rename(index={0:'column type'})
    tab_info=tab_info.append(pd.DataFrame(data.isnull().sum()).T.rename(index={0:'null values (nb)'}))
    tab_info=tab_info.append(pd.DataFrame(data.isnull().sum()/data.shape[0]*100).T.rename(index={0:'null values (%)'}))
    return tab_info

In [6]:
#prints the first data file
print("\n".join(train_x[0].split("\n")[:])) 

 

[Table_Title] 
药明康德（603259） 
公司研究/点评报告 

会计核算变更带来净利润明显增益，看好细胞疗法 CDMO 领先地位 

点评报告/医药 

[Table_Summary] 
一、事件概述 

—药明康德事件点评 

2020 年 06 月 11 日 

2020 年 6 月 10 日，公司发布公告审议通过《关于变更对 JW (Cayman) Therapeutics Co., 
Ltd.会计核算方法的议案》，同意自 2020 年 5 月 22 日起，变更对于公司及其子公司持有
JW (Cayman) Therapeutics 股份的会计核算方法，该股份由以权益法计量的长期股权投资
变更为以公允价值计量且其变动计入当期损益的其他非流动金融资产。在丧失重大影响
之日，药明香港控股所持 JW  Cayman 股份的公允价值与长期股权投资账面价值之间的
差额人民币 35,149.08 万元，计入公司 2020 年 5 月 31 日止的当期合并损益表。 

一、 二、分析与判断 

  对 JW  Cayman 会计核算方法变更带来 3.51 亿投资收益，对 2020 年净利润产

生 11.69%正向影响 

2020  年 5  月 22  日，JW  Cayman  完成 B  轮融资。此轮融资完成后，药明香港控股持
有 JW Cayman  股份有 21.31%下降至 17.27%。同时，JW Cayman  通过股东会决议并完
成其股东协议的修改，药明香港控股不再具有向 JW  Cayman  单独及共同委派董事的相
关权利。2020 年 3 月董事会改组，免除药明香港控股对 JW  Cayman 通过董事会参与经
营的决策权，以增强 JW Cayman 细胞疗法产品开发和生产运营的独立性。 
如果按照我们预测的公司 2020 年 26.43 亿归母净利润来估算的话，此次 3.51 亿元投资
收益将对公司 2020 年归母净利润带来 11.69%的增幅。 

  看好公司细胞/基因疗法 CDMO 全球优势地位，未来有望受益于细胞疗法行业

的快速发展   

细胞疗法进入兑现阶段，公司赋能全球药企细胞疗法研发和生产有望优先享受市场红
利。国内已有 CAR-T 产品实现报产，药明巨诺预计也将于 2020 年实现细胞治疗产品的


## Data preprocessing - pipelined

## Algo1 - Naive base

### Train

In [7]:
# pipeline
text_clf_nb = Pipeline([('vect', CountVectorizer()), # attribute stop_words ("then, the, etc.")
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultinomialNB(alpha=0.01)), # FitPrior=False, a uniform prior will be used
                    ])

In [8]:
text_clf_nb = text_clf_nb.fit(train_x, train_y)

### Test

In [9]:
predicted = text_clf_nb.predict(test_x)
np.mean(predicted == test_y)

0.9230769230769231

In [10]:
text_clf_nb.score(test_x, test_y)

0.9230769230769231

## Algo2 - Support Vector Machines

### Train

In [11]:
# pipeline
text_clf_svm = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf-svm', SGDClassifier(loss='hinge', 
                                               penalty='l2',
                                               alpha=1e-3, 
                                               random_state=42)),
                    ])

In [12]:
text_clf_svm = text_clf_svm.fit(train_x, train_y)

### Test

In [13]:
predicted_svm = text_clf_svm.predict(test_x)
np.mean(predicted_svm == test_y)

0.7692307692307693

In [14]:
text_clf_svm.score(test_x, test_y)

0.7692307692307693

## Grid Search

In [15]:
parameters_nb = {'vect__ngram_range': [(1, 1), (1, 2)],
              'tfidf__use_idf': (True, False),
              'clf__alpha': (1e-2, 1e-3),
             }

In [16]:
gs_clf_nb = GridSearchCV(text_clf_nb, parameters_nb, n_jobs=-1)
gs_clf_nb = gs_clf_nb.fit(train_x, train_y)

C:\Users\idgca\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:670: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


In [17]:
gs_clf_nb.best_params_
gs_clf_nb.best_score_

0.8377777777777776

In [18]:
parameters_svm = {'vect__ngram_range': [(1, 1), (1, 2)],
              'tfidf__use_idf': (True, False),
              'clf-svm__alpha': (1e-2, 1e-3),
             }

In [19]:
gs_clf_svm = GridSearchCV(text_clf_svm, parameters_svm, n_jobs=-1)
gs_clf_svm = gs_clf_svm.fit(train_x, train_y)

C:\Users\idgca\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:670: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


In [20]:
gs_clf_svm.best_params_
gs_clf_svm.best_score_

0.86